[Authenticating to sovereign cloud](https://learn.microsoft.com/en-us/python/api/overview/azure/search-documents-readme?view=azure-python#authenticate-in-a-national-cloud)

In [1]:
from pathlib import Path

import os

from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential, AzureAuthorityHosts

from azure.search.documents import SearchClient

env_directory = Path.cwd() / ".azure"

env_file = next(
    (candidate / ".env" for candidate in env_directory.glob("avcoe-*")),
    None,
)

if env_file is None:
    raise FileNotFoundError(
        "Could not locate an avcoe-* environment directory under .azure"
    )


load_dotenv(dotenv_path=env_file, override=False)


search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]

search_index = os.environ["SEARCH_INDEX_NAME"]

print(f"Using search endpoint {search_endpoint}, and index: {search_index}")

authority_host = (
    AzureAuthorityHosts.AZURE_GOVERNMENT
    if os.getenv("CLOUD_NAME") == "AzureUSGovernment"
    else AzureAuthorityHosts.AZURE_PUBLIC_CLOUD
)

print(f"Using authority host: {authority_host}")

Using search endpoint https://avcoe-demo-ai-search-mcp-search.search.azure.us, and index: avcoe-demo-ai-search-mcp-index-and-vectorize
Using authority host: login.microsoftonline.us


In [ ]:
# Set the correct credential scope for Azure Government
audience = (
    "https://search.azure.us"
    if os.getenv("CLOUD_NAME") == "AzureUSGovernment"
    else "https://search.azure.com"
)

print(f"Using credential scope: {audience}")

credential = DefaultAzureCredential()

# Create SearchClient with the scoped credential
search_client = SearchClient(
    search_endpoint, search_index, credential, audience=audience
)

Using credential scope: https://search.azure.us


In [ ]:
results = search_client.search("test", top=1, include_total_count=True)

In [6]:
import json

for result in results:
    print(json.dumps(dict(result), indent=2, default=str))

{
  "text_vector": [
    -0.006681388,
    0.0022139612,
    -0.020638445,
    -0.008745919,
    0.00012087472,
    0.026409516,
    0.0044141817,
    0.03308747,
    -0.033939388,
    -0.03883106,
    0.00093608134,
    0.02967979,
    -0.024389641,
    0.0036893627,
    -0.0044073113,
    0.016475031,
    0.02374383,
    0.00234965,
    0.011871915,
    0.0020078514,
    -0.024114829,
    -0.017464358,
    -0.013987975,
    -0.021641513,
    0.014729969,
    0.010772664,
    0.038528763,
    -0.03633026,
    0.0024853388,
    0.005685191,
    0.0047989194,
    0.009659672,
    -0.025791187,
    0.011143662,
    -0.0065096295,
    -0.031493556,
    -0.004630597,
    -0.0079008695,
    0.01516967,
    0.0017373326,
    0.030916447,
    0.0037408902,
    0.0023771313,
    -0.035780635,
    0.028099615,
    0.0213667,
    0.0031809588,
    -0.013191017,
    0.0065920735,
    0.0023599556,
    0.017739171,
    0.035835598,
    -0.009934485,
    0.0017218743,
    -0.012119248,
    0.009865